In [113]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

# --- Data Loading and Preprocessing ---

def load_and_preprocess_data(file_path):
    """Loads, preprocesses, and cleans the restaurant dataset."""

    df = pd.read_csv(file_path)
    df.dropna(inplace=True)  # Handle missing values

    # Drop unnecessary columns
    columns_to_drop = [
        'Restaurant ID', 'Country Code', 'Address', 'Locality',
        'Locality Verbose', 'Longitude', 'Latitude', 'Currency',
        'Rating color', 'Rating text', 'Votes', 'Is delivering now',
        'Switch to order menu'
    ]
    df.drop(columns_to_drop, axis=1, inplace=True)

    # Encode binary columns
    label_encoder = LabelEncoder()
    binary_columns = ['Has Table booking', 'Has Online delivery']
    for column in binary_columns:
        df[column + '_encoded'] = label_encoder.fit_transform(df[column])
        df.drop(column, axis=1, inplace=True)  # Drop original

    # Group and one-hot encode cities
    city_counts = df['City'].value_counts()
    major_cities = city_counts[city_counts >= 100].index
    df['Grouped_Cities'] = df['City'].apply(lambda x: x if x in major_cities else 'Others')
    city_encoded = pd.get_dummies(df['Grouped_Cities'], prefix='city')
    df = pd.concat([df, city_encoded], axis=1)
    df.drop(['City', 'Grouped_Cities'], axis=1, inplace=True)

    # Encode top N cuisines
    cuisines_list = df['Cuisines'].str.split(', ', expand=True).stack()
    top_cuisines = cuisines_list.value_counts().head(15).index
    for cuisine in top_cuisines:
        cuisine_col_name = f'cuisine_{cuisine.lower().replace(" ", "_")}'
        df[cuisine_col_name] = df['Cuisines'].apply(
            lambda x: 1 if isinstance(x, str) and cuisine in x else 0
        )
    df.drop('Cuisines', axis=1, inplace=True) # Drop the original 'Cuisines' column

    return df


# --- Recommendation Functions ---

def calculate_similarity_matrix(df):
    """Calculates the cosine similarity matrix."""

    features_for_similarity = [col for col in df.columns if col != 'Restaurant Name']
    return cosine_similarity(df[features_for_similarity])


def get_restaurant_recommendations(df, similarity_matrix, restaurant_name, n_recommendations=5):
    """Recommends restaurants similar to a given restaurant name."""

    matching_indices = df[df['Restaurant Name'] == restaurant_name].index
    if len(matching_indices) == 0:
        return "Restaurant not found."
    idx = matching_indices[0]

    sim_scores = list(enumerate(similarity_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:n_recommendations + 1]  # Exclude itself
    restaurant_indices = [i[0] for i in sim_scores]

    return df.iloc[restaurant_indices][[
        'Restaurant Name', 'Average Cost for two', 'Price range', 'Aggregate rating'
    ]]

def get_recommendations_with_preferences(df, preferences, n_recommendations=5):
    """Recommends restaurants based on user preferences."""

    mask = (
        (df['Price range'].isin(preferences['price_range'])) &
        (df['Aggregate rating'] >= preferences.get('min_rating', 0)) # Use .get() for optional preferences
    )

    for cuisine in preferences.get('cuisines', []):  # Handle case with no cuisines
        cuisine_col = f'cuisine_{cuisine.lower().replace(" ", "_")}'
        if cuisine_col in df.columns:
            mask = mask & (df[cuisine_col] == 1)

    return df[mask].sort_values('Aggregate rating', ascending=False).head(n_recommendations)


# --- Main Execution ---

if __name__ == "__main__":
    file_path = "/content/Dataset .csv"  # Replace with your dataset path
    df = load_and_preprocess_data(file_path)
    similarity_matrix = calculate_similarity_matrix(df)

    # Test restaurant-based recommendations
    test_restaurant = "Le Petit Souffle"
    recommendations = get_restaurant_recommendations(df, similarity_matrix, test_restaurant)
    print(f"\nRecommendations similar to {test_restaurant}:")
    print(recommendations)

    # Test preference-based recommendations
    user_preferences = {
        'price_range': [2, 3],
        'cuisines': ['Italian', 'Chinese'],
        'min_rating': 4.0
    }
    recommendations_pref = get_recommendations_with_preferences(df, user_preferences)
    print("\nRecommendations based on user preferences:")
    print(recommendations_pref)


Recommendations similar to Le Petit Souffle:
       Restaurant Name  Average Cost for two  Price range  Aggregate rating
9             Locavore                  1100            3               4.8
13          Guevarra's                  1000            3               4.2
9172            Apache                  1000            3               4.1
9157        Le Plaisir                  1000            3               4.8
1     Izakaya Kikufuji                  1200            3               4.5

Recommendations based on user preferences:
          Restaurant Name  Average Cost for two  Price range  \
734    ECHOES Koramangala                   950            2   
2483   The Fusion Kitchen                  1000            3   
6997  Echoes Satyaniketan                   600            2   
3310        Spezia Bistro                   900            2   
3337      The Hudson Cafe                   850            2   

      Aggregate rating  Has Table booking_encoded  \
734             